<a href="https://colab.research.google.com/github/vijaygwu/classideas/blob/main/LowRankFact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
# Example of a generic low rank factorization

import numpy as np

# Create a user-item matrix (e.g., user ratings)
# This is a simple example; you should replace this with your data
user_item_matrix = np.array([
    [5, 4, 0, 1, 0],
    [4, 0, 3, 1, 0],
    [1, 0, 0, 5, 4],
    [0, 1, 5, 4, 5]
])

# Perform SVD factorization
U, Sigma, VT = np.linalg.svd(user_item_matrix, full_matrices=False)

# Specify the desired low rank (number of latent factors)
k = 2

# Reconstruct the matrix with a lower rank approximation
U_k = U[:, :k]
Sigma_k = np.diag(Sigma[:k])
VT_k = VT[:k, :]

user_item_matrix_approx = np.dot(U_k, np.dot(Sigma_k, VT_k))

# Now, user_item_matrix_approx contains the low-rank approximation of the original matrix
print("Original User-Item Matrix:")
print(user_item_matrix)
print("\nLow-Rank Approximation (k=2):")
print(user_item_matrix_approx)


Original User-Item Matrix:
[[5 4 0 1 0]
 [4 0 3 1 0]
 [1 0 0 5 4]
 [0 1 5 4 5]]

Low-Rank Approximation (k=2):
[[ 5.41568326  2.90021003  0.96027988  0.79937254 -0.42029949]
 [ 3.33507949  1.81162257  1.24419831  1.4210575   0.72236071]
 [ 0.52868834  0.37944563  2.54831089  3.57012382  3.64807301]
 [ 0.46425301  0.38309582  3.50922726  4.94395384  5.11445954]]


In [74]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Decompose a dense layer using SVD
def decompose_dense_layer(layer, rank):
    weights = layer.weight.data.numpy()
    biases = layer.bias.data.numpy()

    # SVD decomposition
    U, S, Vt = np.linalg.svd(weights, full_matrices=False)

    # Take the first 'rank' components
    U_approx = U[:, :rank]
    S_approx = np.sqrt(S[:rank])  # Taking square root of singular values
    Vt_approx = Vt[:rank, :]

    W1 = torch.tensor(U_approx, dtype=torch.float32)
    W2 = torch.tensor(np.dot(np.diag(S_approx), Vt_approx), dtype=torch.float32)

    return W1, W2, biases

# Create the original model
class OriginalModel(nn.Module):
    def __init__(self):
        super(OriginalModel, self).__init__()
        self.fc1 = nn.Linear(784, 1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define the rank for decomposition
rank = 512

# Create the original model instance
original_model = OriginalModel()
print(original_model)

# Decompose the dense layer
W1, W2, biases = decompose_dense_layer(original_model.fc1, rank)

# Create a new model with decomposed layers
class DecomposedModel(nn.Module):
    def __init__(self, rank):
        super(DecomposedModel, self).__init__()
        self.fc1 = nn.Linear(784, rank, bias=False)
        self.fc2 = nn.Linear(rank, 1024, bias=True)
        self.fc3 = nn.Linear(1024, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Create the new model instance
new_model = DecomposedModel(rank)

# Set weights for the decomposed layers
new_model.fc1.weight.data = W1
new_model.fc2.weight.data = W2
new_model.fc2.bias.data = torch.tensor(biases, dtype=torch.float32)

# Print the new model summary
print(new_model)


OriginalModel(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=10, bias=True)
)
DecomposedModel(
  (fc1): Linear(in_features=784, out_features=512, bias=False)
  (fc2): Linear(in_features=512, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=10, bias=True)
)
